In [295]:
from pandas import *
from statsmodels.api import OLS, add_constant

from docx import Document
from docx.enum.table import WD_ALIGN_VERTICAL, WD_ALIGN_PARAGRAPH


ImportError: cannot import name 'WD_ALIGN_PARAGRAPH' from 'docx.enum.table' (C:\Users\Charles Lee\AppData\Local\Programs\Python\Python311\Lib\site-packages\docx\enum\table.py)

preprocess

In [296]:
data_2 = read_csv("C:/Users/Charles Lee/OneDrive - CUHK-Shenzhen/FIN3080_P3/data2.csv")
data_3 = read_csv("C:/Users/Charles Lee/OneDrive - CUHK-Shenzhen/FIN3080_P3/data3.csv")
data_risk_free = read_csv("C:/Users/Charles Lee/OneDrive - CUHK-Shenzhen/FIN3080_P3/datarisk_free.csv")
data_2 = data_2[(data_2['Market Type'] == 1) | (data_2['Market Type'] == 4) | (data_2['Market Type'] == 64)]
data_3 = data_3[(data_3['Market Type'] == 1) | (data_3['Market Type'] == 4) | (data_3['Market Type'] == 64)]
data_main = concat([data_2, data_3], axis=0,ignore_index=True)
data_main = data_main[['code', 'date', 'return']]
data_main['date'] = data_main['date'] + '-0'
data_main['date'] = to_datetime(data_main['date'], format='%Y-%U-%w')
data_risk_free['trading_date_yw'] = to_datetime(data_risk_free['trading_date_yw'])

In [297]:
data_main.loc[data_main['date'].dt.year == 2018, 'date'] -= Timedelta(days=6)
data_main.loc[data_main['date'].dt.year == 2019, 'date'] -= Timedelta(days=5)
data_main.loc[data_main['date'].dt.year == 2020, 'date'] -= Timedelta(days=4)
data_main.loc[data_main['date'].dt.year == 2021, 'date'] -= Timedelta(days=2)
data_main.loc[data_main['date'].dt.year == 2022, 'date'] -= Timedelta(days=1)

In [298]:
market_return = data_main.groupby('date')['return'].mean().reset_index()
market_return.columns = ['date', 'market_return']

In [299]:
data_main = data_main.merge(market_return, on='date', how='left')
data_main = data_main.merge(data_risk_free, left_on='date', right_on='trading_date_yw', how='left')

In [300]:
data_main.dropna(inplace=True)
data_main = data_main[['code', 'date', 'return', 'market_return', 'risk_free_return']]
data_main['ri_rf'] = data_main['return'] - data_main['risk_free_return']
data_main['rm_rf'] = data_main['market_return'] - data_main['risk_free_return']

In [301]:
data_main = data_main[['code', 'date', 'return', 'ri_rf', 'rm_rf','risk_free_return']]

In [302]:
data_p1 = data_main[data_main['date'] <= '2018-12-31']
data_p2 = data_main[(data_main['date'] >= '2019-01-01') & (data_main['date'] <= '2020-12-31')]
data_p3 = data_main[data_main['date'] >= '2021-01-01']

p1

In [303]:
beta = {}
for code in data_p1.groupby('code').groups.keys():
    data = data_p1[data_p1['code'] == code]
    model = OLS(data['ri_rf'], add_constant(data['rm_rf']))
    results = model.fit()
    beta[code] = results.params['rm_rf']

In [304]:
beta = DataFrame(beta.items(), columns=['code', 'beta'])
beta

,code,beta
0,1,0.330210
1,2,0.626524
2,4,0.581006
3,5,1.015351
4,6,1.179834
...,...,...
2827,603993,1.275410
2828,603996,1.137413
2829,603997,0.897913
2830,603998,1.219376


p2

In [305]:
data_p2 = data_p2.merge(beta, on='code', how='left')
data_p2.dropna(inplace=True)

In [306]:
data_p2['decile'] = qcut(data_p2['beta'], 10, labels=[1,2,3,4,5,6,7,8,9,10])

In [307]:
data_p2_result = DataFrame()
for i in range(1,11):
    data_decile = data_p2[data_p2['decile'] == i]
    grouped_data = data_decile.groupby('date')['return'].mean().reset_index()
    grouped_data.columns = ['date', 'portfolio return']
    grouped_data['decile'] = i
    data_p2_result = concat([data_p2_result, grouped_data], axis=0, ignore_index=True)
data_p2_partial = data_p2[['date', 'risk_free_return', 'rm_rf']].drop_duplicates(subset=['date'])
data_p2_result = data_p2_result.merge(data_p2_partial, on='date', how='left')
data_p2_result = data_p2_result[['decile', 'date', 'portfolio return', 'risk_free_return', 'rm_rf']]

In [308]:
data_p2_result['rp_rf'] = data_p2_result['portfolio return'] - data_p2_result['risk_free_return']
data_p2_result

,decile,date,portfolio return,risk_free_return,rm_rf,rp_rf
0,1,2019-01-01,-0.004267,0.000495,0.016791,-0.004762
1,1,2019-01-08,0.016326,0.000500,0.029782,0.015826
2,1,2019-01-15,0.006844,0.000498,0.001606,0.006346
3,1,2019-01-22,-0.007350,0.000493,-0.005897,-0.007843
4,1,2019-01-29,-0.014185,0.000484,-0.032571,-0.014669
...,...,...,...,...,...,...
1015,10,2020-11-25,-0.013646,0.000585,-0.008693,-0.014231
1016,10,2020-12-02,0.016438,0.000584,0.013317,0.015854
1017,10,2020-12-09,-0.045060,0.000573,-0.038665,-0.045633
1018,10,2020-12-16,-0.006833,0.000549,0.002807,-0.007382


In [309]:
beta2 = {}
alpha = {}
alpha_t = {}
alpha_significance = {}
beta_t = {}
beta_significance = {}
r_squared = {}

# Iterate through each decile
for decile in data_p2_result.groupby('decile').groups.keys():
    data = data_p2_result[data_p2_result['decile'] == decile]
    model = OLS(data['rp_rf'], add_constant(data['rm_rf']))
    results = model.fit()
    
    beta2[decile] = results.params['rm_rf']
    alpha[decile] = results.params['const']
    alpha_t[decile] = results.tvalues['const']
    alpha_significance[decile] = results.pvalues['const']
    beta_t[decile] = results.tvalues['rm_rf']
    beta_significance[decile] = results.pvalues['rm_rf']
    r_squared[decile] = results.rsquared

beta2 = DataFrame(beta2.items(), columns=['decile', 'beta_p'])
alpha = DataFrame(alpha.items(), columns=['decile', 'alpha'])
alpha_t = DataFrame(alpha_t.items(), columns=['decile', 'alpha_t'])
alpha_significance = DataFrame(alpha_significance.items(), columns=['decile', 'alpha_significance'])
beta_t = DataFrame(beta_t.items(), columns=['decile', 'beta_t'])
beta_significance = DataFrame(beta_significance.items(), columns=['decile', 'beta_significance'])
r_squared = DataFrame(r_squared.items(), columns=['decile', 'R_squared'])

table = merge(alpha, alpha_t, on='decile')
table = merge(table, alpha_significance, on='decile')
table = merge(table, beta2, on='decile')
table = merge(table, beta_t, on='decile')
table = merge(table, beta_significance, on='decile')
table = merge(table, r_squared, on='decile')

table.to_excel("C:/Users/Charles Lee/OneDrive - CUHK-Shenzhen/FIN3080_P3/table.xlsx")


p3

In [310]:
data_p3

,code,date,return,ri_rf,rm_rf,risk_free_return
205,1,2021-01-01,0.072062,0.071577,0.004905,0.000485
206,1,2021-01-08,0.026370,0.025888,-0.020070,0.000482
207,1,2021-01-15,0.057935,0.057431,-0.014085,0.000504
208,1,2021-01-22,0.049048,0.048494,0.017571,0.000554
209,1,2021-01-29,0.048116,0.047558,-0.043548,0.000558
...,...,...,...,...,...,...
889065,873527,2022-11-26,-0.016269,-0.016713,-0.010337,0.000444
889066,873527,2022-12-03,-0.005513,-0.005984,0.027869,0.000471
889067,873527,2022-12-10,-0.024390,-0.024867,0.003103,0.000477
889068,873527,2022-12-17,-0.019318,-0.019777,-0.014859,0.000459


In [311]:
data_p3 = data_p3.merge(beta, on='code', how='left')
data_p3.dropna(inplace=True)
data_p3['decile'] = qcut(data_p3['beta'], 10, labels=[1,2,3,4,5,6,7,8,9,10])
data_p3_result = DataFrame({'decile':[], 'portfolio return':[], 'rp_rf':[]})
for i in range(1,11):
    data_decile = data_p3[data_p3['decile'] == i]
    data_p3_result.loc[len(data_p3_result)] = {
        'portfolio return': data_decile['return'].mean(),
        'rp_rf': data_decile['ri_rf'].mean(),
        'decile': i
    }

In [312]:
data_p3_result = data_p3_result[['decile', 'rp_rf']]
data_p3_result

,decile,rp_rf
0,1,0.001210
1,2,0.001663
2,3,0.001244
3,4,0.001413
4,5,0.002132
5,6,0.001666
6,7,0.002379
7,8,0.001631
8,9,0.002309
9,10,0.002397


In [313]:
data_p3_result = data_p3_result.merge(beta2, on='decile', how='left')
data_p3_result

,decile,rp_rf,beta_p
0,1,0.001210,0.799998
1,2,0.001663,0.881876
2,3,0.001244,0.896626
3,4,0.001413,0.947006
4,5,0.002132,1.001360
5,6,0.001666,1.047664
6,7,0.002379,1.038506
7,8,0.001631,1.104073
8,9,0.002309,1.085995
9,10,0.002397,1.149520


In [314]:
models = OLS(data_p3_result['rp_rf'], add_constant(data_p3_result['beta_p']))
results = models.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  rp_rf   R-squared:                       0.549
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     9.757
Date:                Thu, 18 Apr 2024   Prob (F-statistic):             0.0141
Time:                        20:26:27   Log-Likelihood:                 67.091
No. Observations:                  10   AIC:                            -130.2
Df Residuals:                       8   BIC:                            -129.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0013      0.001     -1.281      0.2

C:\Users\Charles Lee\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
